# GUI

In [ ]:
import sys
from PIL import ImageTk
from tkinter.filedialog import askopenfilename
from PIL import Image, ImageFile
from matplotlib.pyplot import imshow
import matplotlib.image
import requests
from io import BytesIO
import numpy as np
import cv2

import numpy as np
from tensorflow.keras.models import load_model
import h5py
from tensorflow.keras.preprocessing import image
model = load_model("C:/Users/sNutesh/Desktop/VT_Class/Major1.0/Model4.h5", compile=False)

from imageai.Detection import ObjectDetection
import os
execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()

try:
    import Tkinter as tk
except ImportError:
    import tkinter as tk

try:
    import ttk
    py3 = False
except ImportError:
    import tkinter.ttk as ttk
    py3 = True

import GUI_support

top = tk.Tk()


def OpenFile():
    global file
    filename = askopenfilename()
    file = filename
    image1 = Image.open(filename)
    image1 = image1.resize((400, 200),Image.ANTIALIAS)
    image1 = ImageTk.PhotoImage(image1)
    Canvas1 = tk.Canvas(top)
    Canvas1.place(relx=0.038, rely=0.257, height=200, width=400)
    Canvas1.configure(background="#1f2833")
    Canvas1.configure(borderwidth="-1")
    Canvas1.configure(insertbackground="black")
    Canvas1.configure(relief="ridge")
    Canvas1.configure(selectbackground="#c4c4c4")
    Canvas1.configure(selectforeground="black")
    Canvas1.create_image(0,0,image=image1,anchor='nw')
    DetectLeaf()
    top.mainloop()

def RemBG():
    global file
    
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 30
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,0.0)
    
    img = cv2.imread(file)
    img = cv2.resize(img  , (1000 ,260 ))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#---------------------------- Edge detection ------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)
    contour_info = []
    contours,_ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]
    
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    mask_stack = np.dstack([mask]*3)
    mask_stack  = mask_stack.astype('float32') / 255.0          
    img         = img.astype('float32') / 255.0 
    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) 
    masked = (masked * 255).astype('uint8')                     
    cv2.imwrite('masked.jpg', masked)
    
    hsv = cv2.cvtColor(masked,cv2.COLOR_BGR2HSV)
    
    lower_range = np.array([21, 8, 63], dtype=np.uint8)  # Works for BACTERIAL LEAF (100% Accuracy)
    upper_range = np.array([30, 255, 255], dtype=np.uint8)
    
    Ylower_range = np.array([17, 100, 100], dtype=np.uint8)  # For detecting yellow in paddy blast
    Yupper_range = np.array([23, 255, 255], dtype=np.uint8)
    
    Blower_range = np.array([0, 80, 40], dtype=np.uint8)  # Works for separating brown spots and bacterial leaf
    Bupper_range = np.array([20, 255, 255], dtype=np.uint8)
    
    maskLB = cv2.inRange(hsv, lower_range, upper_range)
    maskY = cv2.inRange(hsv, Ylower_range, Yupper_range)
    maskB = cv2.inRange(hsv, Blower_range, Bupper_range)
    
    tempY = maskY - maskB
    tempB = maskB - maskY
    
    result = cv2.bitwise_and(img, img, mask=maskB)
    cv2.imwrite("Result.JPG", result*255)
    
    image2 = Image.open('masked.jpg')
    image2 = image2.resize((400, 200),Image.ANTIALIAS)
    image2 = ImageTk.PhotoImage(image2)
    Canvas2 = tk.Canvas(top)
    Canvas2.place(relx=0.346, rely=0.257, height=200, width=400)
    Canvas2.configure(background="#1f2833")
    Canvas2.configure(borderwidth="2")
    Canvas2.configure(insertbackground="black")
    Canvas2.configure(relief="ridge")
    Canvas2.configure(selectbackground="#c4c4c4")
    Canvas2.configure(selectforeground="black")
    Canvas2.create_image(0,0,image=image2,anchor='nw')
    top.mainloop()

def SepDis():
    image3 = Image.open('result.jpg')
    image3 = image3.resize((400, 200),Image.ANTIALIAS)
    image3 = ImageTk.PhotoImage(image3)
    Canvas3 = tk.Canvas(top)
    Canvas3.place(relx=0.654, rely=0.257, height=200, width=400)
    Canvas3.configure(background="#1f2833")
    Canvas3.configure(borderwidth="2")
    Canvas3.configure(insertbackground="black")
    Canvas3.configure(relief="ridge")
    Canvas3.configure(selectbackground="#c4c4c4")
    Canvas3.configure(selectforeground="black")
    Canvas3.create_image(0,0,image=image3,anchor='nw')
    top.mainloop()
    
def make_square(img):
    cols,rows = img.size
    
    if rows>cols:
        pad = (rows-cols)/2
        img = img.crop((pad,0,cols,cols))
    else:
        pad = (cols-rows)/2
        img = img.crop((0,pad,rows,rows))
    
    return img

def DetectLeaf():
    global file
    
    detections = detector.detectObjectsFromImage(input_image=file, output_image_path=os.path.join(execution_path , "imagenew.jpg"))
    t = 0
    for eachObject in detections:
        t = eachObject["percentage_probability"]
    Label7 = tk.Label(top)
    Label7.place(relx=0.615, rely=0.729, height=40, width=400)
    Label7.configure(background="#1f2833")
    Label7.configure(disabledforeground="#000000")
    Label7.configure(font=font16)
    Label7.configure(foreground="#66fcf1")
    if (t!=0):
        Label7.configure(text='''Not a Leaf''')
        Button2["state"] = 'disabled'
        Button3["state"] = 'disabled'
        Button4["state"] = 'disabled'
    else:
        Label7.configure(text='''Leaf''')

def Predict():
    global file
        
    test_image = []
    img = image.load_img(file, grayscale=True)
    img = make_square(img)
    img = img.resize((256,256), Image.ANTIALIAS)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
    test = np.array(test_image)
    prediction = model.predict_classes(test)
    
    Label7 = tk.Label(top)
    Label7.place(relx=0.615, rely=0.729, height=40, width=400)
    Label7.configure(background="#1f2833")
    Label7.configure(disabledforeground="#000000")
    Label7.configure(font=font16)
    Label7.configure(foreground="#66fcf1")
    
    if(prediction[0] == 1):
        Label7.configure(text='''Leaf Smut''')
    if(prediction[0] == 2):
        Label7.configure(text='''Bacterial Blight''')
    if(prediction[0] == 3):
        Label7.configure(text='''Brown Spot''')
    if(prediction[0] == 4):
        Label7.configure(text='''Sheath Blight''')
    if(prediction[0] == 5):
        Label7.configure(text='''Tungro''')
    if(prediction[0] == 6):
        Label7.configure(text='''Healthy''')
    
    top.mainloop()

#-------------------------End of Function-----------------------------------------


#------------------------Begining of GUI------------------------------------------

_bgcolor = '#1f2833'  # X11 color: 'gray85'
_fgcolor = '#66fcf1'  # X11 color: 'black'
_compcolor = '#1f2833' # X11 color: 'gray85'
_ana1color = '#1f2833' # X11 color: 'gray85'
_ana2color = '#ffffff' # Closest X11 color: 'gray92'
font12 = "-family {Times New Roman} -size 13"
font13 = "-family {Times New Roman} -size 12"
font14 = "-family {Times New Roman} -size 14"
font16 = "-family {Times New Roman} -size 13 -weight bold"
font9 = "-family {Times New Roman} -size 24 -slant italic "  \
    "-underline 1"
style = ttk.Style()
if sys.platform == "win32":
    style.theme_use('winnative')
style.configure('.',background=_bgcolor)
style.configure('.',foreground=_fgcolor)
style.configure('.',font="TkDefaultFont")
style.map('.',background=
    [('selected', _compcolor), ('active',_ana2color)])

top.geometry("1300x700+19+10")
top.minsize(120, 1)
top.maxsize(1370, 749)
top.resizable(1, 1)
top.title("GUI")
top.configure(background="#1f2833")

# button0 = Image.open("button.png")
# button0 = button0.resize((200, 40),Image.ANTIALIAS)
# button0 = ImageTk.PhotoImage(button0)
# Canvas0 = tk.Canvas(top)
# Canvas0.place(relx=0.0, rely=0.0, height=700, width=1300)
# Canvas0.create_image(0,0,image=image0,anchor='nw')

Label1 = tk.Label(top)
Label1.place(relx=0.0, rely=0.0, height=80, width=1300)
Label1.configure(background="#1f2833")
Label1.configure(disabledforeground="#000000")
Label1.configure(font=font9)
Label1.configure(foreground="#66fcf1")
Label1.configure(text='''Rice Leaf Disease Detection''')

TSeparator1 = ttk.Separator(top)
TSeparator1.place(relx=0.0, rely=0.114, relwidth=1.0)

Button1 = tk.Button(top, command=OpenFile)
Button1.place(relx=0.125, rely=0.157, height=40, width=200)
Button1.configure(activebackground="#ffffff")
Button1.configure(activeforeground="#66fcf1")
Button1.configure(background="#1f2833")
Button1.configure(disabledforeground="#000000")
Button1.configure(font=font12)
Button1.configure(foreground="#66fcf1")
Button1.configure(highlightbackground="#1f2833")
Button1.configure(highlightcolor="black")
Button1.configure(pady="1")
Button1.configure(text='''Upload Image''')

Canvas1 = tk.Canvas(top)
Canvas1.place(relx=0.038, rely=0.257, height=200, width=400)
Canvas1.configure(background="#1f2833")
Canvas1.configure(borderwidth="2")
Canvas1.configure(insertbackground="black")
Canvas1.configure(relief="ridge")
Canvas1.configure(selectbackground="#c4c4c4")
Canvas1.configure(selectforeground="black")

Button2 = tk.Button(top, command=RemBG, )
Button2.place(relx=0.438, rely=0.157, height=40, width=200)
Button2.configure(activebackground="#ffffff")
Button2.configure(activeforeground="#66fcf1")
Button2.configure(background="#1f2833")
Button2.configure(disabledforeground="#000000")
Button2.configure(font=font12)
Button2.configure(foreground="#66fcf1")
Button2.configure(highlightbackground="#1f2833")
Button2.configure(highlightcolor="black")
Button2.configure(pady="0")
Button2.configure(text='''Remove Background''')

Canvas2 = tk.Canvas(top)
Canvas2.place(relx=0.346, rely=0.257, height=200, width=400)
Canvas2.configure(background="#1f2833")
Canvas2.configure(borderwidth="2")
Canvas2.configure(insertbackground="black")
Canvas2.configure(relief="ridge")
Canvas2.configure(selectbackground="#c4c4c4")
Canvas2.configure(selectforeground="black")

Canvas3 = tk.Canvas(top)
Canvas3.place(relx=0.654, rely=0.257, height=200, width=400)
Canvas3.configure(background="#1f2833")
Canvas3.configure(borderwidth="2")
Canvas3.configure(insertbackground="black")
Canvas3.configure(relief="ridge")
Canvas3.configure(selectbackground="#c4c4c4")
Canvas3.configure(selectforeground="black")

Button3 = tk.Button(top, command=SepDis)
Button3.place(relx=0.731, rely=0.157, height=40, width=200)
Button3.configure(activebackground="#ffffff")
Button3.configure(activeforeground="#66fcf1")
Button3.configure(background="#1f2833")
Button3.configure(disabledforeground="#000000")
Button3.configure(font=font12)
Button3.configure(foreground="#66fcf1")
Button3.configure(highlightbackground="#1f2833")
Button3.configure(highlightcolor="black")
Button3.configure(pady="0")
Button3.configure(text='''Separate Disease''')

Label2 = tk.Label(top)
Label2.place(relx=0.038, rely=0.571, height=30, width=401)
Label2.configure(background="#1f2833")
Label2.configure(disabledforeground="#000000")
Label2.configure(font=font12)
Label2.configure(foreground="#66fcf1")
Label2.configure(text='''Original Leaf Image''')

Label3 = tk.Label(top)
Label3.place(relx=0.346, rely=0.571, height=30, width=400)
Label3.configure(background="#1f2833")
Label3.configure(disabledforeground="#000000")
Label3.configure(font=font12)
Label3.configure(foreground="#66fcf1")
Label3.configure(text='''Background Removed Image''')

Label4 = tk.Label(top)
Label4.place(relx=0.654, rely=0.571, height=30, width=401)
Label4.configure(background="#1f2833")
Label4.configure(disabledforeground="#000000")
Label4.configure(font=font12)
Label4.configure(foreground="#66fcf1")
Label4.configure(text='''Diseased Portion''')

TSeparator2 = ttk.Separator(top)
TSeparator2.place(relx=0.0, rely=0.629, relwidth=1.0)

Button4 = tk.Button(top, command=Predict)
Button4.place(relx=0.238, rely=0.729, height=40, width=200)
Button4.configure(activebackground="#ffffff")
Button4.configure(activeforeground="#66fcf1")
Button4.configure(background="#1f2833")
Button4.configure(disabledforeground="#000000")
Button4.configure(font=font12)
Button4.configure(foreground="#66fcf1")
Button4.configure(highlightbackground="#1f2833")
Button4.configure(highlightcolor="black")
Button4.configure(pady="0")
Button4.configure(text='''Result''')

Label6 = tk.Label(top)
Label6.place(relx=0.538, rely=0.657, height=30, width=202)
Label6.configure(background="#1f2833")
Label6.configure(disabledforeground="#000000")
Label6.configure(font=font12)
Label6.configure(foreground="#66fcf1")
Label6.configure(text='''Classification Result :''')

Label7 = tk.Label(top)
Label7.place(relx=0.615, rely=0.729, height=40, width=400)
Label7.configure(background="#1f2833")
Label7.configure(disabledforeground="#000000")
Label7.configure(font=font16)
Label7.configure(foreground="#66fcf1")


top.mainloop()

W0507 20:27:47.359038  8364 deprecation.py:506] From C:\Users\sNutesh\Anaconda3\envs\class1\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0507 20:27:47.373165  8364 deprecation.py:506] From C:\Users\sNutesh\Anaconda3\envs\class1\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0507 20:27:47.373165  8364 deprecation.py:506] From C:\Users\sNutesh\Anaconda3\envs\class1\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python